# Package

In [1]:
# 模型架構套件
import torch
import torch.nn as nn
import torch.nn.functional as F

# 資料處理套件
from torch.utils.data import Dataset, DataLoader

# 最佳化到件
import torch.optim as optim

# others常見套件
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data Processing Preparation

In [2]:
# 設定隨機種子以確保可重現性
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# 定義數據集
class PressurePumpDataset(Dataset):
    def __init__(self, num_samples=10000, anomaly_ratio=0.05):
        self.num_samples = num_samples
        self.data = []
        self.labels = []
        for _ in range(num_samples):
            A = float(np.random.normal(loc=5.0, scale=1.5))
            B = float(np.random.binomial(1, 0.5))
            
            if A > 8.0:
                label = 1 if B == 1 else 0
            else:
                label = 1
                
            if label == 0 and random.random() > anomaly_ratio:
                label = 1
                
            self.data.append([A, B])
            self.labels.append(label)
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

In [3]:
train_dataset = PressurePumpDataset(num_samples=8000, anomaly_ratio=0.1)
test_dataset = PressurePumpDataset(num_samples=2000, anomaly_ratio=0.1)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model Design

In [4]:
class AttentionLayer(nn.Module):
    def __init__(self, input_dim):
        super(AttentionLayer, self).__init__()
        output_dim = input_dim
        
        # key, query, value
        self.weights = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim, output_dim)) for _ in range(3)]
        )
        self.biases = nn.ParameterList(
            [nn.Parameter(torch.zeros(output_dim)) for _ in range(3)]
        )
        

    def forward(self, x1, x2):
        k = x1@(self.weights[0]) + self.biases[0]
        v = x1@(self.weights[1]) + self.biases[1]
        q = x2@(self.weights[2]) + self.biases[2]
        return (q@(k.T)/(k.shape[-1])**0.5)@(v)

class InteractionLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(InteractionLayer, self).__init__()
        self.interaction = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(self, x):
        # Residual Connection
        return self.interaction(x) + x

class JointAutoencoder(nn.Module):
    def __init__(self):
        super(JointAutoencoder, self).__init__()
        
        # Encoder A: Dealing with Continuous Data
        self.encoder_A = nn.Sequential(nn.Linear(1, 32), nn.ReLU(), nn.Linear(32, 16), nn.ReLU())
        
        # Encoder B: Dealing with Descrete Data
        # Its a status code with 1 bit.
        self.encoder_B = nn.Sequential(nn.Embedding(num_embeddings=2, embedding_dim=4), nn.Linear(4, 16), nn.ReLU())
        
        # Ineraction Layer
        self.interaction = InteractionLayer(input_dim=32, hidden_dim=16)
        
        # Attention Layer
        self.attention = AttentionLayer(input_dim=16)
        
        # Latent Space
        self.latent = nn.Sequential(nn.Linear(16, 8), nn.Tanh())
        
        # Decoder A
        self.decoder_A = nn.Sequential(nn.Linear(8, 16), nn.ReLU(), nn.Linear(16, 1))
        
        # Decoder B
        self.decoder_B = nn.Sequential(nn.Linear(8, 8), nn.ReLU(), nn.Linear(8, 2), nn.ReLU(), nn.Linear(2, 1), nn.Sigmoid())
        
    def forward(self, A, B):
        x_A = self.encoder_A(A)  # [batch_size, 16]
        x_B = self.encoder_B(B.long()).squeeze(1)  # [batch_size, 16]
        attended = self.attention(x_A, x_B)  # [batch_size, 32]
        z = self.latent(attended)  # [batch_size, 16]
        decoded_A = self.decoder_A(z)  # [batch_size, 1]
        decoded_B = self.decoder_B(z)  # [batch_size, 2]
        return decoded_A, decoded_B, z

# Training Process

In [5]:
# 訓練函數
def train_autoencoder(model, loader, criterion_A, criterion_B, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_data, _ in loader:
        A = batch_data[:, 0].unsqueeze(1).to(device)  # [batch_size, 1]
        B = batch_data[:, 1].unsqueeze(1).to(device)  # [batch_size, 1]
        
        optimizer.zero_grad()
        recon_A, recon_B, _ = model(A, B)
        
        # 重建損失
        loss_A = criterion_A(recon_A, A)
        loss_B = criterion_B(recon_B.squeeze(1), B.squeeze(1))
        loss = loss_A + loss_B
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * A.size(0)
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

# 評估函數
def evaluate_autoencoder(model, loader, criterion_A, criterion_B, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for batch_data, _ in loader:
            A = batch_data[:, 0].unsqueeze(1).to(device)
            B = batch_data[:, 1].unsqueeze(1).to(device)
            
            recon_A, recon_B, _ = model(A, B)
            
            loss_A = criterion_A(recon_A, A)
            loss_B = criterion_B(recon_B.squeeze(1), B.squeeze(1))
            loss = loss_A + loss_B
            
            running_loss += loss.item() * A.size(0)
    
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

# Training Object Assign

In [6]:
# 設定設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 初始化模型、損失函數和優化器
model = JointAutoencoder().to(device)
criterion_A = nn.MSELoss().to(device)
criterion_B = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


# 訓練過程
num_epochs = 1000
best_loss = float('inf')

Using device: cuda


In [7]:
for epoch in range(num_epochs):
    train_loss = train_autoencoder(model, train_loader, criterion_A, criterion_B, optimizer, device)
    val_loss = evaluate_autoencoder(model, test_loader, criterion_A, criterion_B, device)
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'best_autoencoder.pth')
    
    if (epoch+1) % 5 == 0 or epoch == 0:
        print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')

# 異常檢測
# 加載最佳模型
model.load_state_dict(torch.load('best_autoencoder.pth'))
model.eval()

# 定義計算重建誤差的函數
def calculate_reconstruction_error(model, loader, device):
    errors = []
    labels = []
    with torch.no_grad():
        for batch_data, batch_labels in loader:
            A = batch_data[:, 0].unsqueeze(1).to(device)
            B = batch_data[:, 1].unsqueeze(1).to(device)
            recon_A, recon_B, _ = model(A, B)
            
            # 計算重建誤差
            loss_A = F.mse_loss(recon_A, A, reduction='none')
            loss_A = loss_A.mean(dim=1)  # [batch_size]
            
            # 對於重建B的部分，使用交叉熵損失
            loss_B = F.cross_entropy(recon_B.squeeze(1), B.squeeze(1), reduction='none')  # [batch_size]
            
            # 總重建誤差
            loss = loss_A + loss_B  # [batch_size]
            
            errors.extend(loss.cpu().numpy())
            labels.extend(batch_labels.numpy())
    return np.array(errors), np.array(labels)

# 計算測試集的重建誤差
errors, labels = calculate_reconstruction_error(model, test_loader, device)

# 繪製重建誤差分佈
plt.figure(figsize=(10,6))
sns.histplot(errors[labels==1], color='green', label='Normal', kde=True, stat="density", bins=50)
sns.histplot(errors[labels==0], color='red', label='Anomaly', kde=True, stat="density", bins=50)
plt.legend()
plt.xlabel('Reconstruction Error')
plt.ylabel('Density')
plt.title('Reconstruction Error Distribution')
plt.show()

Epoch 1/1000 | Train Loss: 152.4264 | Val Loss: 147.4524
Epoch 5/1000 | Train Loss: 133.3960 | Val Loss: 138.4540
Epoch 10/1000 | Train Loss: 133.3963 | Val Loss: 138.4541
Epoch 15/1000 | Train Loss: 133.3962 | Val Loss: 138.4539
Epoch 20/1000 | Train Loss: 133.3963 | Val Loss: 138.4542
Epoch 25/1000 | Train Loss: 133.3967 | Val Loss: 138.4565
Epoch 30/1000 | Train Loss: 133.3990 | Val Loss: 138.4547
Epoch 35/1000 | Train Loss: 133.3969 | Val Loss: 138.4613
Epoch 40/1000 | Train Loss: 133.3980 | Val Loss: 138.4594
Epoch 45/1000 | Train Loss: 133.3984 | Val Loss: 138.4539
Epoch 50/1000 | Train Loss: 133.4005 | Val Loss: 138.4539
Epoch 55/1000 | Train Loss: 133.4006 | Val Loss: 138.4578
Epoch 60/1000 | Train Loss: 133.3976 | Val Loss: 138.4546
Epoch 65/1000 | Train Loss: 133.4010 | Val Loss: 138.4550
Epoch 70/1000 | Train Loss: 133.3982 | Val Loss: 138.4540
Epoch 75/1000 | Train Loss: 133.3990 | Val Loss: 138.4564
Epoch 80/1000 | Train Loss: 133.3978 | Val Loss: 138.4542
Epoch 85/1000 | 


KeyboardInterrupt



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 設定隨機種子以便重現結果
torch.manual_seed(0)
np.random.seed(0)

# 假設每個序列的長度為 100
sequence_length = 100
num_sequences = 5

# 生成五個隨機氣壓時間序列（A、B、C、D、E）
# 假設正常序列來自正態分佈，異常序列有較大偏移
sequences = []
for i in range(num_sequences):
    if i == 4:  # 假設第五個序列是異常的
        seq = np.random.normal(loc=10.0, scale=1.0, size=sequence_length)
    else:
        seq = np.random.normal(loc=0.0, scale=1.0, size=sequence_length)
    sequences.append(seq)

# 將數據轉換為 PyTorch 張量，並進行標準化
sequences = torch.tensor(sequences, dtype=torch.float32)
# 標準化每個序列
sequences = (sequences - sequences.mean(dim=1, keepdim=True)) / sequences.std(dim=1, keepdim=True)

# 添加一個維度以符合 LSTM 的輸入要求 (batch, seq, feature)
sequences = sequences.unsqueeze(-1)  # shape: (5, 100, 1)

class LSTM_Autoencoder(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super(LSTM_Autoencoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 編碼器
        self.encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # 解碼器
        self.decoder = nn.LSTM(hidden_size, input_size, num_layers, batch_first=True)
        
    def forward(self, x):
        # 編碼
        encoded, (hidden, cell) = self.encoder(x)
        
        # 解碼
        decoded, _ = self.decoder(encoded)
        return decoded

# 選擇正常的序列進行訓練（假設前四個是正常的）
train_data = sequences[:4]  # shape: (4, 100, 1)

# 定義模型、損失函數和優化器
model = LSTM_Autoencoder(input_size=1, hidden_size=64, num_layers=2)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練參數
num_epochs = 1000
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(train_data)
    loss = criterion(outputs, train_data)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 評估並檢測異常
model.eval()
with torch.no_grad():
    reconstructed = model(sequences)
    # 計算每個序列的均方誤差
    mse = torch.mean((reconstructed - sequences) ** 2, dim=(1,2))
    print("每個序列的重建誤差（MSE）:", mse.numpy())

    # 假設重建誤差最大的序列為異常
    anomaly_index = torch.argmax(mse).item()
    sequence_names = ['A', 'B', 'C', 'D', 'E']
    print(f"異常序列為: {sequence_names[anomaly_index]}")


C:\Users\jhbai\AppData\Local\Temp\ipykernel_34552\2847560561.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  sequences = torch.tensor(sequences, dtype=torch.float32)


Epoch [200/1000], Loss: 1.0323
Epoch [400/1000], Loss: 0.7619
Epoch [600/1000], Loss: 0.6418
Epoch [800/1000], Loss: 0.5482
Epoch [1000/1000], Loss: 0.4814
每個序列的重建誤差（MSE）: [0.44693908 0.4459344  0.5328663  0.4982229  0.5576994 ]
異常序列為: E
